# WEB SCRAPING 

En este notebook se encuentra el código usado para realizar el escaneo y extracción de información de distintas páginas web para conformar el dataset de precios de vivienda en el Valle de San Nicolas, finalmente se decide usar unicamente los datos obtenidos del sitio Finca Raíz.

www.fincaraiz.com

In [1]:
!pip install selenium
!pip install beautifulsoup4
!pip install lxml

In [4]:
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.set_capability('unhandledPromptBehavior', 'accept')
#chrome_options.add_argument(r"load-extension=C:\Users\Andre\AppData\Local\Google\Chrome\User Data\Default\Extensions\cfhdojbkjhnklbpkdaibdccddilifddb\3.11.1_0");


In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup, Comment
from selenium.webdriver import ActionChains
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
from progressbar import progressbar as pbar

## **Información Página Finca Raiz**

***Capturar URLS de finca raiz***

In [16]:
# Capturar datos de finca raiz
driver = webdriver.Chrome('D:/chromedriver',chrome_options=chrome_options)
URL="https://google.com.co"
driver.get(URL)
driver.maximize_window()
time.sleep(1)

sopas=[]

#Hay 4057 resultados lo que da aprox 91 paginas
#Inicialmente se navega en la URL de búsqueda de finca raíz para capturar todas las URLs de los resultados deseados, posterior a esto,
#se hace scraping sobre cada una de las paaginas para extraer los datos de interés

for i in range(91):
             
    URL2=f"https://www.fincaraiz.com.co/apartamento-casa-finca-casa-campestre-apartaestudio-casa-lote/venta/rionegro-el-retiro-la-ceja-guarne-el-carmen-de-viboral-marinilla-llano-grande-san-antonio-de-pereira-santuario/?ad=30|{i}||||1||8,9,7,21,22,23|||55|5500004,5500013,5500012,5500018,5500026,5500024,5500020,5500022,5500029,5500028||||||||||||||||1|||1||griddate%20desc||||-1||||"   

    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL+"t");
    driver.get(URL2);
    #time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    sopas.append(soup)
    
    #Cerrar la pestaña actual
    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL + "w");
    
    print(f"Páginas escaneadas:{i+1}")

driver.close()

<ipython-input-16-3749573d1fda>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('D:/chromedriver',chrome_options=chrome_options)


Páginas escaneadas:1
Páginas escaneadas:2
Páginas escaneadas:3
Páginas escaneadas:4
Páginas escaneadas:5
Páginas escaneadas:6
Páginas escaneadas:7
Páginas escaneadas:8
Páginas escaneadas:9
Páginas escaneadas:10
Páginas escaneadas:11
Páginas escaneadas:12
Páginas escaneadas:13
Páginas escaneadas:14
Páginas escaneadas:15
Páginas escaneadas:16
Páginas escaneadas:17
Páginas escaneadas:18
Páginas escaneadas:19
Páginas escaneadas:20
Páginas escaneadas:21
Páginas escaneadas:22
Páginas escaneadas:23
Páginas escaneadas:24
Páginas escaneadas:25
Páginas escaneadas:26
Páginas escaneadas:27
Páginas escaneadas:28
Páginas escaneadas:29
Páginas escaneadas:30
Páginas escaneadas:31
Páginas escaneadas:32
Páginas escaneadas:33
Páginas escaneadas:34
Páginas escaneadas:35
Páginas escaneadas:36
Páginas escaneadas:37
Páginas escaneadas:38
Páginas escaneadas:39
Páginas escaneadas:40
Páginas escaneadas:41
Páginas escaneadas:42
Páginas escaneadas:43
Páginas escaneadas:44
Páginas escaneadas:45
Páginas escaneadas:

In [17]:
# los proyectos nuevos tienen una estructura diferente que los usados en la página de detalle 
# A través de Beatiful soup se realiza una transformación del código html de los sitios para extraer los datos a 
# partir de etiquetas, clases y ids de los elementos de la página web

paginas_prop=[]

for soup in sopas:

    propiedades = soup.find_all('ul', class_='advert')

    for data in propiedades:
        url=data.a.get("href")
        

        if url.find("https://") == -1:
            url="https://www.fincaraiz.com.co"+url

        paginas_prop.append(url)

In [18]:

df_paginas_prop = pd.DataFrame (paginas_prop,columns=['URLS'])

# Guardar lista de páginas 
df_paginas_prop.to_csv('PAGINAS_act_08.08.2021.csv', header=True, index=False, sep=';', encoding='latin1')

In [19]:
# Función para quitar comentarios de un documento html, ya que esto genera errores al momento de extraer datos
def quitar_comentarios(datos):
  for element in datos(text=lambda text: isinstance(text, Comment)):
      element.extract()

In [110]:
#paginas_prop=['https://www.fincaraiz.com.co/avanti/el-retiro/proyecto-nuevo-det-4283005.aspx?itemid=4283008','https://www.fincaraiz.com.co/altos-de-santana/rionegro/proyecto-nuevo-det-6351568.aspx?itemid=6357639']

In [20]:
#Limpiar urls duplicadas

import urllib.parse as up
from urllib.parse import parse_qs

clean_url=[]

for url in paginas_prop:
    
    if url.find("http://logs1237.xiti.com/") != -1 or url.find("http://logc237.xiti.com") != -1:
        parsed =  up.urlparse(url)
        parsed= parse_qs(parsed.query)['url'][0]
        clean=parsed.replace('?itemid=','').replace(parse_qs(up.urlparse(parsed).query)['itemid'][0],'')
        clean_url.append(clean)
    else:
        clean_url.append(url)
        
paginas_prop=list(dict.fromkeys(clean_url))

In [21]:
len(paginas_prop)

2727

In [22]:
paginas_prop[:10]

['https://www.fincaraiz.com.co/tao/rionegro/proyecto-nuevo-det-5469246.aspx',
 'https://www.fincaraiz.com.co/finca-en-venta/rionegro/vereda_los_alticos_san_antonio_pereira-det-6280668.aspx',
 'https://www.fincaraiz.com.co/apartamento-en-venta/el-retiro/sierra_grande-det-6508536.aspx',
 'http://www.fincaraiz.com.co/vivari-de-san-nicolas/rionegro/proyecto-nuevo-det-6568602.aspx',
 'https://www.fincaraiz.com.co/apartamento-en-venta/la-ceja/palmazera-det-6252094.aspx',
 'https://www.fincaraiz.com.co/apartamento-en-venta/el-retiro/sector_puro_cuero-det-6570912.aspx',
 'https://www.fincaraiz.com.co/casa-en-venta/la-ceja/jardines_del_camino_urbanizacion_abierta-det-6386948.aspx',
 'https://www.fincaraiz.com.co/casa-en-venta/el-retiro/loma_del_escobero-det-5706019.aspx',
 'https://www.fincaraiz.com.co/casa-en-venta/rionegro/san_antonio_pereira-det-6327008.aspx',
 'https://www.fincaraiz.com.co/altos-de-santana/rionegro/proyecto-nuevo-det-6351568.aspx?itemid=6564216']

In [174]:
#Leer una a una la página de las propiedades y extraer información en un array de diccionarios

from selenium.common.exceptions import TimeoutException

data_detalle=[]


def run_session(url):
    
    driver = webdriver.Chrome('D:/chromedriver',options=chrome_options)
    
    #driver.execute_cdp_cmd('Network.setBlockedURLs', {"urls": ["www.fincaraiz.com.co/webservices/utils.asmx", "https://us.creativecdn.com/","https://tpc.googlesyndication.com/"]})
    
    URL1="https://google.com.co"
    driver.get(URL1)
    driver.maximize_window()

    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL+"t");
    
    #driver.set_page_load_timeout(6)
    

    driver.get(url);
    soup = BeautifulSoup(driver.page_source, 'lxml')


    #Hacer 2 reintentos
    if soup.title.text == '502 Bad Gateway':
        driver.get(url);
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if soup.title.text == '502 Bad Gateway':
            driver.get(url);
            soup = BeautifulSoup(driver.page_source, 'lxml')

    datos_apartamentos = soup.find_all("div", {"id":"typology"})

    try:
        script_data = driver.execute_script('return sfAdvert')
    except:
        script_data ={}
        
    info={}
    
    #Es propiedad usada
    if not datos_apartamentos:

        info["tipo"]='Usada'
        info["url"]=url
        
        try:
            info["titulo"]=soup.find(class_='title').find(class_='box').h1.text.replace('\n','').strip()
        except:
            return []
        
        info["precio"]=soup.find(class_='price').h2.text.replace('\n','').strip()
        info["area"]=soup.find(class_='features').find(class_='advertSurface').text.replace('\n','').strip()
        info["habitaciones"]=soup.find(class_='features').find(class_='advertRooms').text.replace('\n','').replace('Habitaciones:','').strip()
        info["banos"]=soup.find(class_='features').find(class_='advertBaths').text.replace('\n','').replace('Baños:','').strip()
        info["garajes"]=soup.find(class_='features').find(class_='advertGarages').text.replace('\n','').replace('Parqueaderos:','').strip()
        info["descripcion"] = soup.find(class_="description").p.text

        try:
            caractint = soup.find("ul", {"id":"tblInitialInteriores"})
            caractint_clean=[caract.text for caract in caractint]
            for carac in caractint_clean:
                info[carac]=1
        except:
            caractint_clean=[]

        try:
            caractext = soup.find("ul", {"id":"tblInitialExteriores"})
            caractext_clean=[caract.text for caract in caractext]
            for carac in caractext_clean:
                info[carac]=1
        except:        
            caractext_clean=[]

        try:
            caractsec = soup.find("ul", {"id":"tblInitialdelSector"})
            caractsec_clean=[caract.text for caract in caractsec]
            for carac in caractsec_clean:
                info[carac]=1

        except:
            caractsec_clean=[]

        info["caractint"] = caractint_clean
        info["caractext"] = caractext_clean
        info["caractsec"] = caractsec_clean

        otros_datos = soup.find(class_='features_2').ul
        quitar_comentarios(otros_datos)

        otros_datos = [dato.text for dato in otros_datos if dato != '\n']

        datos_estrato = [dato.replace('\n','').replace('Estrato:','').strip() for dato in otros_datos if dato.find("Estrato") != -1 ]
        info["estrato"] = datos_estrato[0] if len(datos_estrato)>0  else ''

        datos_antigu = [dato.replace('\n','').replace('Antigüedad:','').strip() for dato in otros_datos if dato.find("Antigüedad") != -1 ]
        info["antiguedad"] =  datos_antigu[0] if len(datos_antigu)>0  else ''
        info["otros_datos"] = script_data
        
        try:
          info["latitud"] = script_data["Latitude"]
          info["longitud"] = script_data["Longitude"]
        except:
          info["latitud"] = 0
          info["longitud"] = 0

        data_detalle.append(info)

    #Es propiedad nueva
    else:

        datos_apartamentos = soup.find("div", {"id":"typology"}).table.tbody

        array_apa=[]

        
        info_apartamento=[]
        
        for apt in datos_apartamentos:
                
            #info_apartamento=[caract.text.replace('\n','').strip() for caract in apt if caract != '\n']
            
            for caract in apt:
                if caract != '\n':
                    info_apartamento.append(caract.text.replace('\n','').strip())
            
        if len(info_apartamento)>0:
            array_apa.append(info_apartamento)


        for apartamento in array_apa:

            
            info["tipo"]='Nueva'
            info["url"]=url
            
            try:
                info["titulo"]= soup.find(class_='title').h1.text.replace('\n','') +' ' + soup.find(class_='title').span.text.replace('\n','').strip() 
            except:
                return []
            
            info["precio"]= apartamento[6] if len(apartamento[6])>0  else  soup.find(class_='price').h2.text.replace('\n','').strip()
            info["area"]= apartamento[2] if len(apartamento[2])>0  else soup.find(class_='features').find(class_='advertSurface').text.replace('\n','').strip()
            info["habitaciones"]=apartamento[4] if len(apartamento[4])>0  else soup.find(class_='features').find(class_='advertRooms').text.replace('\n','').replace('Habitaciones:','').strip()
            info["banos"]= apartamento[5] if len(apartamento[5])>0  else soup.find(class_='features').find(class_='advertBaths').text.replace('Baños:','').strip()
            info["garajes"]=soup.find(class_='features').find(class_='advertGarages').text.replace('\n','').replace('Parqueaderos:','').strip()
            info["descripcion"] = soup.find(class_="description").b.text + ' ' + soup.find(class_="description").p.text
            info["antiguedad"] = 0


        try:
            caractint = soup.find("ul", {"id":"tblInitialInteriores"})
            caractint_clean=[caract.text for caract in caractint]
            for carac in caractint_clean:
                info[carac]=1
        except:
            caractint_clean=[]

        try:
            caractext = soup.find("ul", {"id":"tblInitialExteriores"})
            caractext_clean=[caract.text for caract in caractext]
            for carac in caractext_clean:
                info[carac]=1

        except:        
            caractext_clean=[]

        try:
            caractsec = soup.find("ul", {"id":"tblInitialdelSector"})
            caractsec_clean=[caract.text for caract in caractsec]
            for carac in caractsec_clean:
                info[carac]=1

        except:
            caractsec_clean=[]

        info["caractint"] = caractint_clean
        info["caractext"] = caractext_clean
        info["caractsec"] = caractsec_clean
        info["otros_datos"] = script_data

        try:
          info["latitud"] = script_data["Latitude"]
          info["longitud"] = script_data["Longitude"]
        except:
          info["latitud"] = 0
          info["longitud"] = 0

    data_detalle.append(info)
        
    return info

    #Cerrar la pestaña actual
    driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL + "w");

    print(f"Página escaneada")

    driver.close() 



In [175]:
from joblib import Parallel, delayed

from tqdm import tqdm

#data_result = Parallel(n_jobs=4)(delayed(run_session)(url) for url in tqdm(paginas_prop))

In [179]:
len(data_detalle)

17

In [177]:
for url in tqdm(paginas_prop[:10]):
    run_session(url)

100%|██████████| 10/10 [01:16<00:00,  7.63s/it]


In [30]:
%debug

> c:\users\andre\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py(242)check_response()
    240                 alert_text = value['alert'].get('text')
    241             raise exception_class(message, screen, stacktrace, alert_text)
--> 242         raise exception_class(message, screen, stacktrace)
    243 
    244     def _value_or_default(self, obj, key, default):



ipdb>  url


*** NameError: name 'url' is not defined


ipdb>  quit


In [ ]:
len(data_result)

In [180]:
import pandas as pd
df_propiedades = pd.DataFrame(data_detalle)


In [181]:
pd.set_option('display.max_columns', None)
df_propiedades.head(2) 

,tipo,url,titulo,precio,area,habitaciones,banos,garajes,descripcion,antiguedad,Balcón,Comedor,Parqueadero Visitantes,Servicios Públicos,Shut de basura,Terraza,Ventilación Natural,Zona de lavandería,Acceso Pavimentado,Asador,Ascensor,Garaje / Parqueadero(s),Gimnasio,Jardín,Piscina,Portería / Recepción,Salón Comunal,Salón de Juegos,Vigilancia privada 24*7,Vista panorámica,Zona de Camping,Área Urbana,Colegios / Universidades,Cómodas vias de acceso,Parques cercanos,Restaurantes,Supermercados / C.Comerciales,Trans. Público cercano,Zona Residencial,caractint,caractext,caractsec,otros_datos,latitud,longitud,Baño Auxiliar,Closet,Cocina Integral,Cuarto de Servicio,Estudio,Lote En construcción,Piso en Baldosa / Mármol,Garaje(s),estrato,Barra estilo americano,Calentador,Depósito / Bodega,Instalación de gas,Circuito cerrado de TV,En conjunto cerrado,Vigilancia,Zonas Verdes,Vivienda Multifamiliar,Zona Infantil,Bombas de gasolina,Cerca centro comercial,Seguridad,Citófono,Cocina tipo Americano,Sobre vía principal,Zona Comercial,Baño Independiente,Canchas Deportivas,Cerca de Zona Urbana,Chimenea,Comedor auxiliar,Hall de Alcobas,Parqueadero interno,Piso de alta resistencia,Piso en cemento,Bahía exterior de parqueo,Cancha de Squash,En condominio,Garaje Cubierto,Oficina de negocios,Planta Eléctrica,Portería / Vigilancia,Senderos ecológicos,Tanques de Agua,Sobre vía secundaria,Zona Campestre
0,Nueva,https://www.fincaraiz.com.co/tao/rionegro/proy...,TAO Vía El Tranvía - Rionegro,$ 294.000.000,"53,00 m2",2,2,1,5469246 Lanzamiento TAO Torre 2. El precio inc...,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"[Balcón, Comedor, Parqueadero Visitantes, Serv...","[Acceso Pavimentado, Asador, Ascensor, Garaje ...","[Área Urbana, Colegios / Universidades, Cómoda...","{'Address': 'Transversal 46 C/C 59', 'Administ...",6.16023540496826,-75.3661499023438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Usada,https://www.fincaraiz.com.co/finca-en-venta/ri...,Finca en Venta Rionegro VEREDA LOS ALTICOS SAN...,$ 950.000.000,"330,00 m²",4,5,2,4 casas en construcción en parcelación en el C...,Menos de 1 año,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,"[Baño Auxiliar, Closet, Cocina Integral, Cuart...",[Garaje(s)],"[Cómodas vias de acceso, Supermercados / C.Com...",{'Address': 'VEREDA LOS ALTICOS SAN ANTONIO DE...,6.12991523742676,-75.3797073364258,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_propiedades["otros_datos"][0]

In [ ]:
print(f'Número de registros: {df_propiedades.shape[0]}')

print(f'Número de Columnas: {df_propiedades.shape[1]}')

print('Columnas:')

list(df_propiedades.columns)

In [ ]:
#Guardar datos en rchivo csv para uso posterior
df_propiedades.to_csv('consolidado_propiedades_FR_29052021.csv', header=True, index=False, sep=';', encoding='latin1')

# ESTE ES EL CODIGO QUE FUNCIONA RAPIDO

In [ ]:
from bs4 import BeautifulSoup
import concurrent.futures
import time
import requests
import re
import json

In [258]:
data_detalle=[]

MAX_THREADS = 30

def download_url(url):
    
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text, 'lxml')

    datos_apartamentos = soup.find_all("div", {"id":"typology"})

    pattern = re.compile("var sfAdvert = \"(.*?)\";")
        
    json_string= soup.find_all('script')
    data_javascript=''

    for d in json_string:
        for data in d:
            data_javascript+=data

    start=data_javascript.find('sfAdvert = {"AdvertId"')
    end=data_javascript.find('MapFR.latitude')

    var_data=data_javascript[start:end].replace('\r','').replace('\n','').replace('sfAdvert =','').replace(';','').strip()
    script_data=json.loads(var_data)
    
    info={}
    
    #Es propiedad usada
    if not datos_apartamentos:

        info["tipo"]='Usada'
        info["url"]=url
        
        try:
            info["titulo"]=soup.find(class_='title').find(class_='box').h1.text.replace('\n','').strip()
        except:
            return []
        
        info["precio"]=soup.find(class_='price').h2.text.replace('\n','').strip()
        info["area"]=soup.find(class_='features').find(class_='advertSurface').text.replace('\n','').strip()
        info["habitaciones"]=soup.find(class_='features').find(class_='advertRooms').text.replace('\n','').replace('Habitaciones:','').strip()
        info["banos"]=soup.find(class_='features').find(class_='advertBaths').text.replace('\n','').replace('Baños:','').strip()
        info["garajes"]=soup.find(class_='features').find(class_='advertGarages').text.replace('\n','').replace('Parqueaderos:','').strip()
        info["descripcion"] = soup.find(class_="description").p.text

        try:
            caractint = soup.find("ul", {"id":"tblInitialInteriores"})
            caractint_clean=[caract.text for caract in caractint]
            for carac in caractint_clean:
                info[carac]=1
        except:
            caractint_clean=[]

        try:
            caractext = soup.find("ul", {"id":"tblInitialExteriores"})
            caractext_clean=[caract.text for caract in caractext]
            for carac in caractext_clean:
                info[carac]=1
        except:        
            caractext_clean=[]

        try:
            caractsec = soup.find("ul", {"id":"tblInitialdelSector"})
            caractsec_clean=[caract.text for caract in caractsec]
            for carac in caractsec_clean:
                info[carac]=1

        except:
            caractsec_clean=[]

        info["caractint"] = caractint_clean
        info["caractext"] = caractext_clean
        info["caractsec"] = caractsec_clean

        otros_datos = soup.find(class_='features_2').ul
        quitar_comentarios(otros_datos)

        otros_datos = [dato.text for dato in otros_datos if dato != '\n']

        datos_estrato = [dato.replace('\n','').replace('Estrato:','').strip() for dato in otros_datos if dato.find("Estrato") != -1 ]
        info["estrato"] = datos_estrato[0] if len(datos_estrato)>0  else ''

        datos_antigu = [dato.replace('\n','').replace('Antigüedad:','').strip() for dato in otros_datos if dato.find("Antigüedad") != -1 ]
        info["antiguedad"] =  datos_antigu[0] if len(datos_antigu)>0  else ''
        info["otros_datos"] = script_data
        
        try:
          info["latitud"] = script_data["Latitude"]
          info["longitud"] = script_data["Longitude"]
        except:
          info["latitud"] = 0
          info["longitud"] = 0

        data_detalle.append(info)

    #Es propiedad nueva
    else:

        datos_apartamentos = soup.find("div", {"id":"typology"}).table.tbody

        array_apa=[]

        
        info_apartamento=[]
        
        for apt in datos_apartamentos:
                
            #info_apartamento=[caract.text.replace('\n','').strip() for caract in apt if caract != '\n']
            
            for caract in apt:
                if caract != '\n':
                    info_apartamento.append(caract.text.replace('\n','').strip())
            
        if len(info_apartamento)>0:
            array_apa.append(info_apartamento)


        for apartamento in array_apa:

            
            info["tipo"]='Nueva'
            info["url"]=url
            
            try:
                info["titulo"]= soup.find(class_='title').h1.text.replace('\n','') +' ' + soup.find(class_='title').span.text.replace('\n','').strip() 
            except:
                return []
            
            info["precio"]= apartamento[6] if len(apartamento[6])>0  else  soup.find(class_='price').h2.text.replace('\n','').strip()
            info["area"]= apartamento[2] if len(apartamento[2])>0  else soup.find(class_='features').find(class_='advertSurface').text.replace('\n','').strip()
            info["habitaciones"]=apartamento[4] if len(apartamento[4])>0  else soup.find(class_='features').find(class_='advertRooms').text.replace('\n','').replace('Habitaciones:','').strip()
            info["banos"]= apartamento[5] if len(apartamento[5])>0  else soup.find(class_='features').find(class_='advertBaths').text.replace('Baños:','').strip()
            info["garajes"]=soup.find(class_='features').find(class_='advertGarages').text.replace('\n','').replace('Parqueaderos:','').strip()
            info["descripcion"] = soup.find(class_="description").b.text + ' ' + soup.find(class_="description").p.text
            info["antiguedad"] = 0


        try:
            caractint = soup.find("ul", {"id":"tblInitialInteriores"})
            caractint_clean=[caract.text for caract in caractint]
            for carac in caractint_clean:
                info[carac]=1
        except:
            caractint_clean=[]

        try:
            caractext = soup.find("ul", {"id":"tblInitialExteriores"})
            caractext_clean=[caract.text for caract in caractext]
            for carac in caractext_clean:
                info[carac]=1

        except:        
            caractext_clean=[]

        try:
            caractsec = soup.find("ul", {"id":"tblInitialdelSector"})
            caractsec_clean=[caract.text for caract in caractsec]
            for carac in caractsec_clean:
                info[carac]=1

        except:
            caractsec_clean=[]

        info["caractint"] = caractint_clean
        info["caractext"] = caractext_clean
        info["caractsec"] = caractsec_clean
        info["otros_datos"] = script_data

        try:
          info["latitud"] = script_data["Latitude"]
          info["longitud"] = script_data["Longitude"]
        except:
          info["latitud"] = 0
          info["longitud"] = 0

    data_detalle.append(info)
    
    #with open(title, "wb") as fh:
    #    fh.write(resp.content)
        
    time.sleep(0.25)
    
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(download_url, story_urls)

def main(story_urls):
    t0 = time.time()
    download_stories(story_urls)
    t1 = time.time()
    print(f"{t1-t0} seconds to download {len(story_urls)} pages.")

In [259]:
main(paginas_prop)

181.56912684440613 seconds to download 2727 pages.


In [260]:
len(data_detalle)

5311

In [284]:
df_propiedades = pd.DataFrame(data_detalle)

In [304]:
print(f'Número de registros: {df_propiedades.shape[0]}')

print(f'Número de Columnas: {df_propiedades.shape[1]}')

print('Columnas:')

#list(df_propiedades.columns)

Número de registros: 5311
Número de Columnas: 182
Columnas:


In [ ]:
df_propiedades.head(50)

In [305]:
#Guardar datos en rchivo csv para uso posterior
df_propiedades.to_csv('consolidado_propiedades_FR_12082021.csv', header=True, index=False, sep=';', encoding='utf_32')

In [290]:
otro_dataset=pd.DataFrame(list(df_propiedades["otros_datos"].to_numpy()))

In [291]:
otro_dataset.drop_duplicates(inplace=True)

In [292]:
len(otro_dataset)

2619

In [293]:
otro_dataset.head()

,AdvertId,AdvertWASId,AdvertExternalId,ClientId,ClientName,ClientURL,AdvertCategoryId,AdvertType,TransactionId,TransactionType,Category1Id,Category2Id,Category3Id,Category1,Category2,Category3,OriginId,Location1Id,Location2Id,Location3Id,Location4Id,Location1,Location2,Location3,Location4,Title,Description,Price,FormatedPrice,ContractType,OutStanding,TopAdvert,ProductsCode,ProductLabel,PhotoUrl,NumPhotos,SemanticText,GridDate,Keywords,LocationType,Status,Phone,Phone2,Phone3,Email1,Email2,Email3,Schedule,Financing,Surface,Area,LivingArea,Address,FormatedSurface,FormatedLivingArea,Rooms,Capacity,Baths,AgeId,Ages,DeliveryDate,ConditionId,Condition,PriceM2,FormatedPriceM2,FloorId,Floor,AdministrationPrice,Neighborhood,QualityRate,WeatherId,Weather,StratumId,Stratum,ViewMapTypeId,ViewMapType,GaragesId,Garages,ParentLevel,GroupType,Extras,ExtrasId,PromotionLogoURL,ClientLogoURL,ClientParentLevel,Latitude,Longitude,GroupName,NumVideos,ModifyDate,PhoneWhatsApp,PhoneClickToCall,PriceTypeId,PriceType,Advertising,TotalEnvironment,InteriorFloors,IncludesAdministration,IsProjectWithOutProduct,Ranking,ProyectStatus,IsProjectWithOutProductNew
0,6568602,6568602,VIVARI DE SAN NICOLAS,3150123,ARQUITECTURA Y CONCRETO SAS,constructora-arquitectura-y-concreto-sas-31501...,2,Used,1,Venta,1,0,0,Proyecto Nuevo,,,23,55,5500004,0,0,Antioquia,Rionegro,,,VIVARI DE SAN NICOLAS,"Vivari San Nicolás, es un conjunto residencial...",319998000,Desde $ 319.998.000 Hasta $ 526.104.820,Professional,False,0,OA,,2021/08/06/6568602/proyecto-nuevo-venta-antioq...,19,/vivari-de-san-nicolas/rionegro/proyecto-nuevo...,8/6/2021 5:23:50 PM,,-1,Active,,,,,,,Lunes a Viernes : 9 a.m - 5 p.m | Sábados y Do...,,55.0,55.0,0,Carrera 56 42 15,"55,66 a 91,97 m2","52,42 a 84,54 m2",1,0,2,0,,,0,,5749155.59,5.749.156/m²,0,,0,San Nicolás,0,0,Templado,0,5,1,,0,1,0,2,"Interiores$Balcón, Baño Auxiliar, Comedor, Est...","32,121,211,122,175,193,134,12,164,103,17,112,1...",2021/08/06/6568602/proyecto-nuevo-venta-antioq...,3150123_p.jpg,0,6.14759492874146,-75.3790435791016,,1,8/6/2021 5:23:50 PM,,,0,,,0,0,False,False,0,0,False
1,6386948,6386948,3991363,1871955,Urve Inmobiliaria,inmobiliaria-urve-inmobiliaria-1871955.aspx,2,Used,1,Venta,9,0,0,Casa,,,23,55,5500012,0,0,Antioquia,La Ceja,,,Casa en Venta - La Ceja Jardines Del Camino Ur...,Venta casa Urbanización Jardines del Camino ...,534000000,$ 534.000.000,Professional,False,0,FA,,2021/06/02/6386948/casa-venta-antioquia-la-cej...,26,/casa-en-venta/la-ceja/jardines_del_camino_urb...,8/10/2021 2:30:47 PM,,-1,Active,,,,,,,,,182.0,182.0,0,Urbanización abierta Jardines del camino,"182,00 m2","182,00 m2",4,0,4,2,1 a 8 años,,0,,2934065.93,2.934.066/m²,2,2º,0,Jardines Del Camino Urbanización Abierta,0,0,,0,4,1,,0,2,0,0,"Interiores$Balcón, Baño Auxiliar, Baño Indepen...","32,121,181,180,20,131,133,134,156,102,164,109,...",,1871955_p.jpg,0,6.02253437042236,-75.4215927124023,,1,8/10/2021 2:30:47 PM,,,0,,,0,0,False,False,0,0,False
3,6575226,0,,3305366,Lotes antioquia,lotes-antioquia-3305366.aspx,2,Used,1,Venta,7,0,0,Finca,,,44,55,5500018,0,0,Antioquia,Guarne,,,Finca en Venta - Guarne guarne,Casa campestre en jurisdicción del municipio d...,270000000,$ 270.000.000,Particular,False,0,,,2021/08/09/6575226/finca-venta-antioquia-guarn...,6,/finca-en-venta/guarne/guarne-det-6575226.aspx,8/9/2021 10:25:23 PM,,-1,Active,,,,,,,,,4070.0,4070.0,0,vereda,"4.070,00 m2","4.070,00 m2",2,0,1,2,1 a 8 años,,0,,66339.07,66.339/m²,0,,0,guarne,0,0,,0,2,1,,0,9,0,0,Exteriores$Terraza,10,,,0,6.23703908920288,-75.3870010375977,,0,8/9/2021 10:56:57 PM,,,0,,,0,0,False,False,0,0,False
5,5730990,5730990,815-1116,128391,D.C COLOMBIA S.A.S.,inmobiliaria-dc-colombia-sas-128391.aspx,2,Used,1,Venta,7,0,0,Finca,,,23,55,5500004,0,0,Antioquia,Rionegro,,,Finca en Venta - Rionegro RIONEGRO,815-1116 D.C COLOMBIA S.A.S vende finca de un ...,1389621713,$ 1.389.621.713,Professional,False,0,FA,,2020/09/17/5730990/finca-venta-antioquia-rione...,20,/finca-en-venta/rionegro/rionegro-det-5730990....,8/

In [297]:
#Guardar datos en rchivo csv para uso posterior
otro_dataset.to_csv('consolidado_propiedades_FRv2_12082021.csv', header=True, index=False, sep=';', encoding='utf_32')